In [1]:
import numpy as np
import pandas as pd

import json
import time
import os
import websocket
import urllib
import hmac
import hashlib

import logging

In [2]:
api_key, api_secret = open('bitmex_api.txt').read().split()
endpoint = "wss://www.bitmex.com/realtime"
symbol = "XBTUSD"

In [3]:
def generate_signature(secret, verb, url, nonce, data):
    """Generate a request signature compatible with BitMEX."""
    # Parse the url so we can remove the base and extract just the path.
    parsedURL = urllib.parse.urlparse(url)
    path = parsedURL.path
    if parsedURL.query:
        path = path + '?' + parsedURL.query

    # print "Computing HMAC: %s" % verb + path + str(nonce) + data
    message = (verb + path + str(nonce) + data).encode('utf-8')

    signature = hmac.new(secret.encode('utf-8'), message, digestmod=hashlib.sha256).hexdigest()
    return signature

def get_auth():
    expires = int(round(time.time() + 3600))
    return [
        "api-expires: " + str(expires),
        "api-signature: " + generate_signature(api_secret, 'GET', '/realtime', expires, ''),
        "api-key:" + api_key
    ]

def get_wsURL(endpoint, symbol):
    symbolSubs = ["quote", "trade"]
    subscriptions = [sub + ':' + symbol for sub in symbolSubs]
    urlParts = list(urllib.parse.urlparse(endpoint))
    urlParts[0] = urlParts[0].replace('http', 'ws')
    urlParts[2] = f"/realtime?subscribe={','.join(subscriptions)}"

    return urllib.parse.urlunparse(urlParts)

In [4]:
def on_message(ws, message):
    now = pd.Timestamp.utcnow()
    message = json.loads(message)
    table = message.get("table")

    if not table:
        return None
    
    action = message.get("action")
    header = action == "partial"
    if not header and action != "insert":
        logging.debug(now, message)
    
    if table == "trade":
        data = pd.DataFrame.from_records(
            data=message.get("data"),
            index="timestamp",
            exclude=["symbol","tickDirection","trdMatchID","grossValue","homeNotional","foreignNotional"]
        )
        data["received"] = now
        data.to_csv(trade_file_name, mode='a', header=header)
        
    elif table == "quote":
        data = pd.DataFrame.from_records(
            data=message.get("data"),
            index="timestamp",
            exclude=["symbol"]
        )
        data["received"] = now
        data.to_csv(quote_file_name, mode='a', header=header)
        
    else:
        logging.debug(now, table)
        
    return None

In [5]:
def collect_data(endpoint, symbol):
    ws = websocket.WebSocketApp(
        url=get_wsURL(endpoint, symbol),
        header=get_auth(),
        on_message=on_message
    )
    try:
        ws.run_forever()
    except:
        pass
    
    return None

In [6]:
os.makedirs("logs", exist_ok=True)
os.makedirs("data", exist_ok=True)
start_time_str = pd.Timestamp.utcnow().strftime("%y%m%d_%H%M")
logger_file_name = f"logs/{symbol}_{start_time_str}.log"
trade_file_name = f"data/{symbol}_trades_{start_time_str}.csv"
quote_file_name = f"data/{symbol}_quotes_{start_time_str}.csv"
logging.basicConfig(filename=logger_file_name,level=logging.DEBUG)
collect_data(endpoint, symbol)

191217_1411
